In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import FinanceLib as fl

%matplotlib inline
%load_ext autoreload
%autoreload 2

D:\Projects\finance-venv\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [3]:
#activate logger
today_dt = fl.date.today().strftime('%Y%m%d')    
log_dir = fl.LOG_DIR + '/logs/' + today_dt

dt_to = fl.date.today()
dt_from = fl.date.today() + fl.datetime.timedelta(days=-2)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
fl.logging.basicConfig(filename=log_dir + '/app.log', filemode='a+', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%H:%M:%S', level=fl.logging.DEBUG)
logger = fl.logging.getLogger()

In [13]:
#load stock list
# fl.save_sp500_tickers()

In [14]:
os.environ['PG_PWD']

'postgres'

In [16]:
stock_df = fl.get_stock_list_from_db()

C:\Projects\FinanceProject\FinanceLib.py:256: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [19]:
stock_df[stock_df['stock'] == "AAPL"]

,stock,security,sector,subindustry,exchange,loaddt,is_load,screener,is_avaible_for_trade
13,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",NASDAQ,2023-06-22 22:08:57.954990,1,america,1


In [21]:
fl.save_stock_quote_to_db

<function FinanceLib.save_stock_quote_to_db(stock, screener, date_from, date_to)>

In [22]:
#check what load in work
fl.save_stock_quote_to_db("AAPL", "america", "2018-01-01", "2023-06-27")

[*********************100%***********************]  1 of 1 completed


0

In [ ]:
fl.get_all_stock_by_period("2018-01-01", "2023-01-01", logger)

C:\Projects\FinanceProject\FinanceLib.py:219: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


SWKS
[*********************100%***********************]  1 of 1 completed
SEDG
[*********************100%***********************]  1 of 1 completed
SBUX
[*********************100%***********************]  1 of 1 completed
STLD
[*********************100%***********************]  1 of 1 completed
SNPS
[*********************100%***********************]  1 of 1 completed
TMUS
[*********************100%***********************]  1 of 1 completed
TROW
[*********************100%***********************]  1 of 1 completed
TTWO
[*********************100%***********************]  1 of 1 completed
TER
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
TXN
[*********************100%***********************]  1 of 1 completed
TSCO
[*********************100%***********************]  1 of 1 completed
TRMB
[*********************100%***********************]  1 of 1 completed
CSX
[*********************100%**********

In [16]:
fl.get_all_stock_recomendation(logger)

C:\Projects\FinanceProject\FinanceLib.py:219: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


SWKS
SEDG
SBUX
STLD
SNPS
TMUS
TROW
TTWO
TER
TSLA
TXN
TSCO
TRMB
CSX
ULTA
UAL
VRSN
VRSK
VRTX
VTRS
WBA
WBD
WTW
WYNN
XEL
ZBRA
ZION
ATVI
ADBE
ADP
AKAM
ALGN
LNT
GOOGL
GOOG
AMZN
AAL
AEP
AMGN
ADI
ANSS
APA
AAPL
AMAT
ACGL
ADSK
AXON
BKR
TECH
BIIB
BKNG
AVGO
CHRW
CDNS
CZR
CDW
CHTR
CINF
CTAS
CSCO
CME
CTSH
CMCSA
CEG
CPRT
AUDCAD
AUDCHF
CSGP
COST
XRAY
DXCM
FANG
DLTR
EBAY
EA
ENPH
EQIX
ETSY
EVRG
NVDA
NXPI
ORLY
ODFL
ON
PCAR
PANW
PARA
PAYX
PYPL
PEP
POOL
PFG
EXPE
EXPD
FFIV
FAST
FITB
FSLR
FTNT
FOXA
FOX
GEHC
GEN
GILD
HAS
HSIC
HOLX
HON
HST
HBAN
IDXX
ILMN
PODD
INTC
INTU
ISRG
JBHT
JKHY
KDP
KLAC
KHC
LRCX
LKQ
MKTX
MAR
MTCH
BRK.B
PRU
TDG
TRV
TFC
TYL
AUDCNH
AMD
TSN
BF.B
USB
UDR
CI
UNP
UPS
URI
UNH
UHS
VLO
VZ
VFC
VICI
V
VMC
WAB
WMT
WM
WAT
WEC
WFC
WELL
WST
ALLE
ALL
MO
AMCR
AEE
AXP
AIG
AMT
AWK
AMP
ABC
AME
APH
AON
APTV
ANET
AJG
AIZ
T
ATO
AZO
BR
BRO
BG
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CVX
AUDCZK
AUDDKK
CMG
CB
CHD
C
CFG
CLX
CMS
KO
CL
CMA
CAG
COP
ED
STZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
E

{'State': 3, 'CntError': 0, 'lst_error': []}

In [5]:
import yfinance as yf

In [60]:
fl.save_stock_quote_to_db('EURUSD', 'Forex', '2023-06-27', '2023-06-28')

CALL finance.p_load_forex('2023-06-28 00:00:00', 'EURUSD', 1.0960105657577515, 1.0962508916854858, 1.0957703590393066, 1.0961307287216187, 1.0961307287216187, 0)
CALL finance.p_load_forex('2023-06-28 01:00:00', 'EURUSD', 1.0958904027938843, 1.0963709354400635, 1.0955302715301514, 1.0957703590393066, 1.0957703590393066, 0)
CALL finance.p_load_forex('2023-06-28 02:00:00', 'EURUSD', 1.0957703590393066, 1.0958904027938843, 1.0949305295944214, 1.0949305295944214, 1.0949305295944214, 0)
CALL finance.p_load_forex('2023-06-28 03:00:00', 'EURUSD', 1.09505033493042, 1.0952903032302856, 1.0945708751678467, 1.0952903032302856, 1.0952903032302856, 0)
CALL finance.p_load_forex('2023-06-28 04:00:00', 'EURUSD', 1.0952903032302856, 1.095650315284729, 1.095170259475708, 1.0954102277755737, 1.0954102277755737, 0)
CALL finance.p_load_forex('2023-06-28 05:00:00', 'EURUSD', 1.0955302715301514, 1.095650315284729, 1.0949305295944214, 1.095170259475708, 1.095170259475708, 0)
CALL finance.p_load_forex('2023-06-

0

In [14]:
stock_df = fl.get_stock_list_from_db()
for ind, row in stock_df.iterrows():
    print(row)

C:\Projects\FinanceProject\FinanceLib.py:226: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


stock                                                 AKAM
security                                            Akamai
sector                              Information Technology
subindustry             Internet Services & Infrastructure
exchange                                            NASDAQ
loaddt                          2023-06-22 22:08:57.954990
is_load                                                  1
screener                                           america
is_avaible_for_trade                                     1
Name: 0, dtype: object
stock                                         ALGN
security                          Align Technology
sector                                 Health Care
subindustry                   Health Care Supplies
exchange                                    NASDAQ
loaddt                  2023-06-22 22:08:57.954990
is_load                                          1
screener                                   america
is_avaible_for_trade                  

In [6]:
import pandas_ta

In [24]:
# df = fl.save_stock_quote_to_db("AAPL", 'america', '2020-01-01', '2023-06-29')

In [57]:
df = fl.get_stock_quote_from_db("EURUSD", 'Forex', is_dt_index = 1, is_stock_index = 0, date_from = None, date_to = None)

select * from finance.f_get_quote (NULL, NULL, 'EURUSD', 'Forex')


C:\Projects\FinanceProject\FinanceLib.py:216: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, parse_dates='Dt', index_col = lst_index)


In [53]:
df = fl.get_stock_quote_from_db("AAPL", 'america', is_dt_index = 1, is_stock_index = 0, date_from = None, date_to = None)

select * from finance.f_get_quote (NULL, NULL, 'AAPL', 'america')


C:\Projects\FinanceProject\FinanceLib.py:216: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, parse_dates='Dt', index_col = lst_index)


In [54]:
df.head()

,stock,openvalue,highvalue,lowvalue,closevalue,adjclose,volume
dt,,,,,,,
2018-04-11,AAPL,43.057499,43.480000,42.924999,43.110001,41.040901,89726400.0
2018-04-12,AAPL,43.352501,43.750000,43.259998,43.535000,41.445507,91557200.0
2018-04-13,AAPL,43.695000,43.959999,43.462502,43.682499,41.585926,100497200.0
2018-04-16,AAPL,43.757500,44.047501,43.707500,43.955002,41.845352,86313600.0
2018-04-17,AAPL,44.122501,44.735001,44.102501,44.560001,42.421310,106421600.0


In [39]:
import pandas as pd

In [41]:
# df.set_index(pd.DatetimeIndex(df["dt"]), inplace=True)

In [56]:
df.ta.log_return(cumulative=True, append=True)

dt
2018-04-11    0.000000
2018-04-12    0.009810
2018-04-13    0.013193
2018-04-16    0.019411
2018-04-17    0.033082
                ...   
2023-06-22    1.467354
2023-06-23    1.465641
2023-06-26    1.458059
2023-06-27    1.473006
2023-06-28    1.479314
Name: CUMLOGRET_1, Length: 1381, dtype: float64

In [43]:
df.ta.percent_return(cumulative=True, append=True)

dt
2018-04-11    0.000000
2018-04-12    0.009858
2018-04-13    0.013280
2018-04-16    0.019601
2018-04-17    0.033635
                ...   
2023-06-22    3.337741
2023-06-23    3.330318
2023-06-26    3.297611
2023-06-27    3.362329
2023-06-28    3.389933
Name: CUMPCTRET_1, Length: 1381, dtype: float64

In [62]:
dir(df.ta)

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_prefix_suffix',
 '_adjusted',
 '_append',
 '_check_na_columns',
 '_constructor',
 '_cores',
 '_df',
 '_dir_additions',
 '_dir_deletions',
 '_exchange',
 '_get_column',
 '_hidden_attrs',
 '_indicators_by_category',
 '_last_run',
 '_mp_worker',
 '_post_process',
 '_reset_cache',
 '_strategy_mode',
 '_time_range',
 '_validate',
 'aberration',
 'above',
 'above_value',
 'accbands',
 'ad',
 'adjusted',
 'adosc',
 'adx',
 'alma',
 'amat',
 'ao',
 'aobv',
 'apo',
 'aroon',
 'atr',
 'bbands',
 'below',
 'below_value',
 'bias',
 'bop',
 'brar',
 'categories',
 'cci',
 'cdl_patt

In [44]:
dir(pandas_ta)

['ALL_PATTERNS',
 'AllStrategy',
 'AnalysisIndicators',
 'BasePandasObject',
 'CANGLE_AGG',
 'CDL_PATTERN_NAMES',
 'Category',
 'CommonStrategy',
 'DataFrame',
 'DistributionNotFound',
 'EXCHANGE_TZ',
 'Imports',
 'List',
 'Optional',
 'PandasObject',
 'Path',
 'Pool',
 'RATE',
 'Series',
 'Strategy',
 'Timedelta',
 'Timestamp',
 'Tuple',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_dist',
 'aberration',
 'above',
 'above_value',
 'accbands',
 'ad',
 'adosc',
 'adx',
 'alma',
 'alphavantage',
 'amat',
 'ao',
 'aobv',
 'apo',
 'argmax',
 'argmin',
 'aroon',
 'atr',
 'av',
 'bbands',
 'below',
 'below_value',
 'bias',
 'bop',
 'brar',
 'cagr',
 'calmar_ratio',
 'candle_color',
 'candles',
 'category_files',
 'cci',
 'cdl',
 'cdl_doji',
 'cdl_inside',
 'cdl_pattern',
 'cdl_z',
 'cfo',
 'cg',
 'chop',
 'cksp',
 'cmf',
 'cmo',
 'combination',
 'coppock',
 'core',
 'cpu_count',
 'cross',
 'cr

In [69]:
df = fl.get_stock_quote_from_db("EURSEK", 'Forex', is_dt_index = 1, is_stock_index = 0, date_from = None, date_to = None)
df.ta.rsi()

select * from finance.f_get_quote (NULL, NULL, 'EURSEK', 'Forex')


C:\Projects\FinanceProject\FinanceLib.py:216: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, parse_dates='Dt', index_col = lst_index)


dt
2023-06-28 00:00:00          NaN
2023-06-28 01:00:00          NaN
2023-06-28 02:00:00          NaN
2023-06-28 03:00:00          NaN
2023-06-28 04:00:00          NaN
2023-06-28 05:00:00          NaN
2023-06-28 06:00:00          NaN
2023-06-28 07:00:00          NaN
2023-06-28 08:00:00          NaN
2023-06-28 09:00:00          NaN
2023-06-28 10:00:00          NaN
2023-06-28 11:00:00          NaN
2023-06-28 12:00:00          NaN
2023-06-28 13:00:00          NaN
2023-06-28 14:00:00    75.407572
2023-06-28 15:00:00    60.737830
2023-06-28 16:00:00    57.907279
2023-06-28 17:00:00    49.185277
2023-06-28 18:00:00    48.571485
2023-06-28 19:00:00    49.902245
2023-06-28 20:00:00    51.113761
2023-06-28 21:00:00    52.722605
2023-06-28 22:00:00    49.072636
2023-06-29 00:00:00    49.066707
2023-06-29 01:00:00    47.804052
2023-06-29 02:00:00    48.714231
2023-06-29 03:00:00    51.242743
2023-06-29 04:00:00    50.926405
2023-06-29 05:00:00    53.689973
2023-06-29 06:00:00    56.990253
2023-06

In [4]:
df_pos = fl.get_freedom_open_position()

In [5]:
df_pos.head()

,mkt_id,q,go,Yield,profit_price,profit_close,i,acc_pos_id,curr,instr_id,...,base_contract_code,name,base_currency,name2,scheme_calc,issue_nb,ltr,acd,fv,s
0,30000000001,10,0,0,6.46,-10.00,ABCL.US,108241055,USD,40002580269,...,ABCL.US,AbCellera Biologics,USD,AbCellera Biologics,T2,CA00288U1066,FIX,0,100,74.60
1,30000000001,5,0,0,7.33,-1.14,AMTX.US,108039906,USD,40000092673,...,AMTX.US,Aemetis Inc,USD,Aemetis Inc,T2,US00770K2024,FIX,0,100,37.79
2,30000000001,1,0,0,115.56,-1.73,ATO.US,108252318,USD,40000056728,...,ATO.US,Atmos Energy Corp.,USD,Atmos Energy Corp.,T2,US0495601058,FIX,0,100,117.29
3,30000000001,1,0,0,53.72,0.47,BSX.US,108108131,USD,40000019667,...,BSX.US,Boston Scientific,USD,Boston Scientific,T2,US1011371077,FIX,0,100,53.25
4,30000000001,2,0,0,49.71,4.66,CARR.US,108241061,USD,40000174546,...,CARR.US,Carrier Global Corp,USD,Carrier Global Corp,T2,US14448C1045,FIX,0,100,94.76


In [6]:
df_pos['stock'] = df_pos['i'].apply(lambda x: x.split('.')[0])

In [7]:
df_rec = fl.get_recomendation_from_db()

D:\Projects\FinanceProject\FinanceLib.py:233: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [8]:
df_rec.head()

,stock,security,exchange,screener,rec_1d,rec_1w,rec_1m,buy_count_1d,sell_count_1d,neutral_count_1d,buy_count_1w,sell_count_1w,neutral_count_1w,buy_count_1m,sell_count_1m,neutral_count_1m,closevalue
0,AKAM,Akamai,NASDAQ,america,BUY,BUY,NEUTRAL,15,1,10,12,4,10,8.0,9.0,9.0,89.870003
1,ALGN,Align Technology,NASDAQ,america,BUY,BUY,BUY,14,3,9,12,5,9,12.0,5.0,9.0,353.640015
2,LNT,Alliant Energy,NASDAQ,america,BUY,SELL,NEUTRAL,11,6,9,5,12,9,8.0,9.0,9.0,52.480000
3,GOOGL,Alphabet Inc. (Class A),NASDAQ,america,NEUTRAL,BUY,STRONG_BUY,8,8,10,13,3,10,15.0,1.0,10.0,119.699997
4,GOOG,Alphabet Inc. (Class C),NASDAQ,america,NEUTRAL,BUY,BUY,8,8,10,13,3,10,13.0,2.0,9.0,120.970001


In [8]:
import pandas as pd
df = pd.merge(df_pos, df_rec,  how='left', left_on=['stock'], right_on = ['stock'])

In [9]:
df.head(20)

,mkt_id,q,go,Yield,profit_price,profit_close,i,acc_pos_id,curr,instr_id,...,buy_count_1d,sell_count_1d,neutral_count_1d,buy_count_1w,sell_count_1w,neutral_count_1w,buy_count_1m,sell_count_1m,neutral_count_1m,closevalue
0,30000000001,10,0,0,6.46,-10.00,ABCL.US,108241055,USD,40002580269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000000001,5,0,0,7.33,-1.14,AMTX.US,108039906,USD,40000092673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000000001,1,0,0,115.56,-1.73,ATO.US,108252318,USD,40000056728,...,17.0,1.0,8.0,17.0,0.0,9.0,15.0,1.0,10.0,116.339996
3,30000000001,1,0,0,53.72,0.47,BSX.US,108108131,USD,40000019667,...,14.0,2.0,10.0,15.0,3.0,8.0,15.0,3.0,8.0,54.090000
4,30000000001,2,0,0,49.71,4.66,CARR.US,108241061,USD,40000174546,...,14.0,3.0,9.0,13.0,2.0,9.0,11.0,0.0,6.0,49.709999
5,30000000001,1,0,0,88.39,2.13,CSGP.US,108241030,USD,40000097804,...,12.0,4.0,10.0,14.0,5.0,7.0,16.0,3.0,7.0,89.000000
6,30000000001,3,0,0,26.66,2.67,DKNG.US,108040239,USD,40000180805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,30000000001,1,0,0,58.99,1.54,FAST.US,108252304,USD,40000019770,...,14.0,4.0,8.0,16.0,4.0,6.0,16.0,1.0,9.0,58.990002
8,30000000001,1,0,0,125.30,3.79,LEN.US,108241132,USD,40000019863,...,14.0,3.0,9.0,14.0,5.0,7.0,16.0,2.0,8.0,125.309998
9,30000000001,1,0,0,188.16,9.43,MMC.US,108108107,USD,40000019893,...,14.0,5.0,7.0,14.0,6.0,6.0,15.0,3.0,8.0,188.080002


In [20]:
df.columns

Index(['mkt_id', 'q', 'go', 'Yield', 'profit_price', 'profit_close', 'i',
       'acc_pos_id', 'curr', 'instr_id', 'k', 't', 'vm', 'mkt_price',
       'accruedint_a', 'bal_price_a', 'price_a', 'face_val_a', 'close_price',
       'market_value', 'currval', 'open_bal', 'base_contract_code', 'name',
       'base_currency', 'name2', 'scheme_calc', 'issue_nb', 'ltr', 'acd', 'fv',
       's', 'stock', 'security', 'exchange', 'screener', 'rec_1d', 'rec_1w',
       'rec_1m', 'buy_count_1d', 'sell_count_1d', 'neutral_count_1d',
       'buy_count_1w', 'sell_count_1w', 'neutral_count_1w', 'buy_count_1m',
       'sell_count_1m', 'neutral_count_1m', 'closevalue'],
      dtype='object')

In [10]:
df = df.rename(columns={'s': 'balance_value', 'q': 'count', 'price_a': 'in_price'})

In [11]:
columns_show = ['stock', 'name', 'count', 'in_price', 'balance_value', 'mkt_price', 'profit_price', 'profit_close', 'market_value', 'rec_1d', 'rec_1w','rec_1m']

In [12]:
df[columns_show].head(20)

,stock,name,count,in_price,balance_value,mkt_price,profit_price,profit_close,market_value,rec_1d,rec_1w,rec_1m
0,ABCL,AbCellera Biologics,10,7.4600,74.60,6.46,6.46,-10.00,64.60,NaN,NaN,NaN
1,AMTX,Aemetis Inc,5,7.5581,37.79,7.33,7.33,-1.14,36.70,NaN,NaN,NaN
2,ATO,Atmos Energy Corp.,1,117.2900,117.29,115.56,115.56,-1.73,116.34,STRONG_BUY,STRONG_BUY,BUY
3,BSX,Boston Scientific,1,53.2500,53.25,53.72,53.72,0.47,54.06,BUY,BUY,BUY
4,CARR,Carrier Global Corp,2,47.3800,94.76,49.71,49.71,4.66,99.40,BUY,BUY,STRONG_BUY
5,CSGP,CoStar Group Inc,1,86.2600,86.26,88.39,88.39,2.13,89.00,BUY,BUY,BUY
6,DKNG,DraftKings Inc,3,25.7700,77.31,26.62,26.66,2.67,79.68,NaN,NaN,NaN
7,FAST,Fastenal Co,1,57.4500,57.45,58.99,58.99,1.54,58.99,BUY,BUY,STRONG_BUY
8,LEN,Lennar Corp.,1,121.5100,121.51,125.30,125.30,3.79,125.32,BUY,BUY,BUY
9,MMC,Marsh & McLennan,1,178.7300,178.73,188.16,188.16,9.43,188.02,BUY,BUY,BUY


In [35]:
from pandasgui import show

In [20]:
def get_pos_and_rec():
    df_pos = fl.get_freedom_open_position()
    df_pos['stock'] = df_pos['i'].apply(lambda x: x.split('.')[0])
    df_rec = fl.get_recomendation_from_db()
    df = pd.merge(df_pos, df_rec,  how='left', left_on=['stock'], right_on = ['stock'])
    columns_show = ['stock', 'name', 'count', 'in_price', 'balance_value', 'mkt_price', 'profit_price', 'profit_close', 'market_value', 'rec_1d', 'rec_1w','rec_1m']
    return df[columns_show]

In [36]:
gui = show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [13]:
# jupyter nbextension enable --py qgrid --sys-prefix
# jupyter nbextension enable --py widgetsnbextension --sys-prefix
# jupyter nbextension install --py bamboolib --sys-prefix
# jupyter nbextension enable --py bamboolib --sys-prefix

SyntaxError: invalid syntax (852628464.py, line 1)

In [18]:
# bam.enable()
bam.disable()

The bamboolib extension was disabled. You can enable it again via 'bam.enable()'. In case that bamboolib was not helpful to you, we are sorry and would like to fix this. Please write us a quick mail to info@8080labs.com so that we can serve you better in the future. Best regards, Tobias and Florian


In [17]:
import bamboolib as bam
import pandas as pd
result = df[columns_show]
bam.show(result)

In [22]:
df = fl.get_pos_and_rec()

In [23]:
df.head(20)

,stock,name,count,in_price,balance_value,mkt_price,profit_price,profit_close,market_value,rec_1d,rec_1w,rec_1m
0,ABCL,AbCellera Biologics,10,7.4600,74.60,6.46,6.46,-10.00,64.60,NaN,NaN,NaN
1,AMTX,Aemetis Inc,5,7.5581,37.79,7.33,7.33,-1.14,36.70,NaN,NaN,NaN
2,ATO,Atmos Energy Corp.,1,117.2900,117.29,115.56,115.56,-1.73,116.34,STRONG_BUY,STRONG_BUY,BUY
3,BSX,Boston Scientific,1,53.2500,53.25,53.72,53.72,0.47,54.06,BUY,BUY,BUY
4,CARR,Carrier Global Corp,2,47.3800,94.76,49.71,49.71,4.66,99.40,BUY,BUY,STRONG_BUY
5,CSGP,CoStar Group Inc,1,86.2600,86.26,88.39,88.39,2.13,89.00,BUY,BUY,BUY
6,DKNG,DraftKings Inc,3,25.7700,77.31,26.62,26.66,2.67,79.68,NaN,NaN,NaN
7,FAST,Fastenal Co,1,57.4500,57.45,58.99,58.99,1.54,58.99,BUY,BUY,STRONG_BUY
8,LEN,Lennar Corp.,1,121.5100,121.51,125.30,125.30,3.79,125.32,BUY,BUY,BUY
9,MMC,Marsh & McLennan,1,178.7300,178.73,188.16,188.16,9.43,188.02,BUY,BUY,BUY
